# ANN Forward and Back Propagation: Computation speed "for-loop" vs. vectorized:
Bellow, there are two implementations of Andrew Ng's Coursera Intro Machine Learning homework#5 about using Artificial Neural Networks for handwritten digit recognition using a set of data from MNIST. The first code follows the exact homework instructions (only in Python instead of MATLAB) which performs forward and back propagation on the network to calculate the cost function and the gradient vector iterating through the entire training set with a for-loop. The second code is the vectorized implementation of the same, without using a for-loop. When I wrote both codes I also attempted to answer the question: which code will run faster? Here, it looks like the two functions perform more or less similarly, however, that seems to be the case only for Jupyter (I wonder why?). Running both codes on other compliers or IDEs like Spyder on my computer, the vectorized implementation was a lot faster (4 to 6 times faster), as Andrew also mentioned in one of the earlier lessons. 

First, let's import the libraries, modules and functions we'll need:

In [2]:
import numpy as np
from numpy import genfromtxt as gft
import scipy.optimize as opt
import time

We'll start with importing the data and preparing them for the calculations:
A quick note about our y vector: The data was taken directly from that of the homework which was modified to match MATLAB indexing (which starts with one instead of zero) and therefore, 10 was used to represent 0 so that other classes are represented by the same number as their labels, we are now turning 10s back to 0s for the same purpose.

In [3]:
read = lambda name : gft(name, delimiter = ',') #A function for reading our CSV source files
X = read('Xmatrix.txt')
y = read('ymatrix.txt')

y[:500]= 0 #First 500 samples were all 10s representing zeros

We'll now implement our sigmoid, sigmoid gradient and forward propagation functions which will be later used in our cost function. There is also a second forward propagation function defined bellow (fpropX) which takes a matrix rather than a vector and can be used for the vectorized cost function as well as testing the optimization results.

In [4]:
m  = np.shape(X)[0] #number of training examples

sigmoid = lambda z: 1.0 / (1+np.exp(-z))

sigmoidGradient = lambda z : sigmoid(z) * (1-sigmoid(z))



def fprop(Theta1,Theta2,X): #Forward Propagation Function, X is a vector
    
    z2 = Theta1 @ X
    
    a2 = sigmoid(z2)
    
    a2 = np.vstack(((np.ones((1,np.shape(a2)[1]))), a2))
    
    z3 = Theta2 @ a2
    
    a3 = sigmoid(z3)
    
    return a3, a2, z2


def fpropX(Theta1,Theta2,X): #Forward Propagation Function, X is a matrix
    
    z2 = Theta1 @ X.T
    
    a2 = sigmoid(z2)
    
    a2 = np.vstack(((np.ones((1,np.shape(a2)[1]))), a2))
    
    z3 = Theta2 @ a2
    
    a3 = sigmoid(z3)
    
    return a3, a2, z2

 # Calculating the cost by iterating through the training set using a for-loop:


In [5]:

def nnCostFxn(nn_params,
              ils,#input layer size
              hls,#hidden layer size
              num_labels,# number of labels (or output classes) 
              X,y,Lambda):#Lambda is our regularization parameter
    
    #These cost functions take the parameters unrolled into a vector
    Theta1 = nn_params[:(hls*(ils+1))].reshape((hls,ils+1))
    Theta2 = nn_params[(hls*(ils+1)):].reshape((num_labels,hls+1))
    
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    Del1 = np.zeros(Theta1.shape)
    Del2 = np.zeros(Theta2.shape)
       
    m = np.shape(X)[0]
    
    J = 0 
    
    ytemp = np.zeros((m,num_labels))
    
    for i in range(m):
        ytemp[i,int(y[i])] = 1
        
    y = ytemp
 
    X = np.hstack((np.ones((m,1)),X))
    
    for i in range(m):
        
        a1 = X[i].reshape((-1,1))
        a3 , a2 , z2 = fprop(Theta1,Theta2,a1)  
        h = a3
        J = J +(1.0/m)*((-y[i,:] @ np.log(h)) - ((1-y[i,:]) @ np.log(1-h)))
        
        del3 = a3 - y[i].reshape((-1,1))
        
        del2 = (Theta2[:,1:].T @ del3) * sigmoidGradient(z2)
        
        
        Del2 = Del2 + del3 @ a2.T
        Del1 = Del1 + del2 @ a1.T
        

    J = J + (Lambda/(2*m)) * (np.sum(np.sum(Theta1[:,1:]**2)) + np.sum(np.sum(Theta2[:,1:]**2)))

        
    Theta1[:,0] = 0
    Theta2[:,0] = 0
    
    Theta1_grad = Del1/m + (Lambda/m)*Theta1
    Theta2_grad = Del2/m + (Lambda/m)*Theta2
    
    grad = np.vstack((Theta1_grad.reshape((-1,1)),Theta2_grad.reshape((-1,1))))
    
    return J , grad

# Vectorized implementation of the Cost function:


In [6]:

def nnCostFxnVec(nn_params,
              ils,#input layer size
              hls,#hidden layer size
              num_labels,
              X,y,Lambda):
    Theta1 = nn_params[:(hls*(ils+1))].reshape((hls,ils+1))
    Theta2 = nn_params[(hls*(ils+1)):].reshape((num_labels,hls+1))
    
    m = np.shape(X)[0]
    
    J = 0 
    
    ytemp = np.zeros((m,num_labels))
    
    for i in range(m):
        ytemp[i,int(y[i])] = 1
        
    y = ytemp
    
    X = np.hstack((np.ones((m,1)),X))

    a3, a2, z2 = fpropX(Theta1,Theta2,X)    
    h = a3
    
    for i in range(m):
        Jsum = (-y[i,:] @ np.log(h[:,i])) - ((1-y[i,:]) @ np.log(1-h[:,i]))
        J = J +(1.0/m)*Jsum
    
    J = J + (Lambda/(2*m)) * (np.sum(np.sum(Theta1[:,1:]**2)) + np.sum(np.sum(Theta2[:,1:]**2)))
    
    del3= a3 - y.T
    
    del2 = (Theta2[:,1:].T @ del3) * sigmoidGradient(z2)
    
    delta2 = del3 @ a2.T
    delta1 = del2 @ X
    
    Theta1[:,0] = 0
    Theta2[:,0] = 0
    
    Theta1_grad = delta1/m + (Lambda/m)*Theta1
    Theta2_grad = delta2/m + (Lambda/m)*Theta2
    
    grad = np.vstack((Theta1_grad.reshape((-1,1)),Theta2_grad.reshape((-1,1))))


    
    return J , grad


# Calculating parameters with fmin_tnc, using both implementations of the cost function and timing each process separately: 

In [7]:
#Parameters to be used for calling the functions:
ils = 400 #input layer size
hls = 25 #hidden layer size
num_labels = 10 # number of lables
Lambda = 1

#random initialization of Theta0 values
np.random.seed(1)
initial_Theta1 = np.random.uniform(-0.12,0.12,(hls,ils+1))
initial_Theta2 = np.random.uniform(-0.12,0.12,(num_labels,hls+1))
nn_params = np.vstack((initial_Theta1.reshape((-1,1)),initial_Theta2.reshape((-1,1)))) 

## Timing the optimization using the for-loop cost function:

In [8]:
start = time.time()
result = opt.fmin_tnc(func=nnCostFxn, x0=nn_params, args=(ils,hls,num_labels,X,y,Lambda))
result_params = result[0]
end = time.time()

## Timing the optimization using the vectorized cost function:

In [9]:
startV = time.time()

resultV = opt.fmin_tnc(func=nnCostFxn, x0=nn_params, args=(ils,hls,num_labels,X,y,Lambda))
resultV_params = resultV[0]

endV = time.time()


# Printing out the results:
Other than the time, we also calculated and printed the accuracy of predictions for each set of calculated parameters which turn out, as expected, to be exactly the same:

In [10]:
Theta1 = result_params[:hls*(ils+1)].reshape((hls,ils+1))
Theta2 = result_params[hls*(ils+1):].reshape((num_labels,hls+1))
X = np.hstack((np.ones((m,1)),X))
h = fpropX(Theta1,Theta2,X)[0]
pred = h.argmax(axis=0)

acc = pred == y
    
percacc = np.mean(acc) *100

print("Time for fmin_tnc to calculate(for-loop cost fxn):", end- start, "\n Accuracy:", percacc)


Theta1V = resultV_params[:hls*(ils+1)].reshape((hls,ils+1))
Theta2V = resultV_params[hls*(ils+1):].reshape((num_labels,hls+1))

hV = fpropX(Theta1V,Theta2V,X)[0]
predV = hV.argmax(axis=0)

accV = predV == y
    
percaccV = np.mean(accV) *100

print("Time for fmin_tnc to calculate(vectorized cost fxn):", endV- startV, "\n Accuracy:", percaccV)

Time for fmin_tnc to calculate(for-loop cost fxn): 296.913743019104 
 Accuracy: 94.84
Time for fmin_tnc to calculate(vectorized cost fxn): 283.21832609176636 
 Accuracy: 94.84
